# Validation of Experiment 7.1: Fine Tuning

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
from pathlib import Path
SCRIPT_DIR = os.path.dirname(os.path.abspath("__init__.py"))
SRC_DIR = Path(SCRIPT_DIR).parent.parent.absolute()
print(SCRIPT_DIR)
print(SRC_DIR)
sys.path.append(os.path.dirname(SRC_DIR))
sys.path.append(os.path.dirname(str(SRC_DIR) + '/models'))

/Users/yannis/PycharmProjects/WeRec3D/src/experiments_evaluation/ex7.1_emc_fine_tune
/Users/yannis/PycharmProjects/WeRec3D/src


In [9]:
from src.models.model2 import create_model
from src.data_loaders.loading import get_val_sets
from src.experiments_evaluation.validation_helpers import calc_total_errors, scale_t2m_back, scale_slp_back, reshape_for_modelling, get_median_pred_days

In [5]:
EXP_NAME = "ex7.1_emc_fine_tune"

In [6]:
F = 5
H = 32
W = 64
CH = 5  # t2m, msl, msk1, msk2, elev
BS = 4
PERCENTAGE = "99"
MISSING_MECHANISM = "mnar"

MODEL_PATH = str(SRC_DIR) + f"/experiments_evaluation/{EXP_NAME}/model_checkpoint/p{PERCENTAGE}/"
MODEL_PATH

'/Users/yannis/PycharmProjects/WeRec3D/src/experiments_evaluation/ex7.1_emc_fine_tune/model_checkpoint/p99/'

In [7]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = create_model(f=F, h=H, w=W, ch=CH, bs=BS)
    model.compile(optimizer=tf.keras.optimizers.Adam(), run_eagerly=None)
    model.load_weights(MODEL_PATH)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Input image batch shape: (4, 5, 32, 64, 5)
Number output channels: 2
Layer1: (4, 5, 32, 64, 64)
Layer2: (4, 5, 16, 32, 128)
Layer3: (4, 5, 16, 32, 128)
Layer4: (4, 5, 8, 16, 256)
Layer5: (4, 5, 8, 16, 256)
Layer6: (4, 5, 8, 16, 256)
Layer7: (4, 5, 8, 16, 256)
Layer8: (4, 5, 8, 16, 256)
Layer9: (4, 5, 8, 16, 256)
Layer10: (4, 5, 8, 16, 256)
Layer11: (4, 5, 8, 16, 256)
Layer12: (4, 5, 8, 16, 256)
Layer13: (4, 5, 16, 32, 128)
Layer14: (4, 5, 16, 32, 128)
Layer15: (4, 5, 32, 64, 64)
Layer16: (4, 5, 32, 64, 32)
Layer17: (4, 5, 32, 64, 2)


In [14]:
val_x, val_y = get_val_sets(variant='b', percentage=PERCENTAGE, include_elevation=True, missing_type=MISSING_MECHANISM)
print(val_x.shape)
print(val_y.shape)

Loading validation variables file: validation.npy
Loading validation mask file: validation_mask_99p.npy
Creating x and y validation sets...
(3653, 40, 68, 5)
(3653, 40, 68, 4)


In [15]:
x = reshape_for_modelling(val_x, seq_shift_reshape=True)
y = reshape_for_modelling(val_y, seq_shift_reshape=False)
print(x.shape)
print(y.shape)

(3646, 5, 32, 64, 5)
(3650, 32, 64, 4)


In [16]:
# Only need variables
x = x[:3644]
y = y[:3644, ..., :2]
print(x.shape, y.shape)

(3644, 5, 32, 64, 5) (3644, 32, 64, 2)


In [17]:
pred = model.predict(x, batch_size=BS)

2024-02-29 08:32:17.036400: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


911/911 [==============================] - 397s 435ms/step


In [18]:
pred = get_median_pred_days(pred)
pred.shape

(3644, 32, 64, 2)

## Error on Climatology

In [19]:
total_error, t2m_error, msl_error = calc_total_errors(y, pred)
print(f"Total error: {total_error}")
print(f"t2m error: {t2m_error}")
print(f"msl error: {msl_error}")

Total error: 0.1536312028869613
t2m error: 0.12881241524231551
msl error: 0.17844999053160632


In [20]:
print(f"t2m scaled: {scale_t2m_back(t2m_error, for_error=True)}")
print(f"msl scaled: {scale_slp_back(msl_error, for_error=True)}")

t2m scaled: 1.145824948492554
msl scaled: 195.94040945358063


## Error on anomaly

In [22]:
from src.config import TA_MU, TA_SIGMA, SLP_MU, SLP_SIGMA

In [24]:
unit_val_ymean = np.load("../../data_sets/ymean_sets/validation_ymean.npy")

normalized_gt = np.load("../../data_sets/variant_b/validation.npy")
normalized_gt = normalized_gt[:3644, 6:38, 2:66]

normalized_val_ymean = (unit_val_ymean - np.array([TA_MU, SLP_MU])) / np.array([TA_SIGMA, SLP_SIGMA])
normalized_val_ymean = normalized_val_ymean[:3644]

In [25]:
normalized_pred = pred
    
norm_pred_anom = normalized_pred - normalized_val_ymean
norm_gt_anom = normalized_gt - normalized_val_ymean
    
normalized_anomaly_mae = np.mean(np.abs(norm_pred_anom - norm_gt_anom)) 
print(f"{normalized_anomaly_mae}")

0.1536312028869613
